In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
        
import warnings
warnings.filterwarnings('ignore')

random_state=20

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [37]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OrdinalEncoder

from sklearn.impute import SimpleImputer

Dataset exploration

In [38]:
df=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

In [39]:
df.head(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [40]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [41]:
df_col_list=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported']

def value_counts(df1,df2,cols):
    for col in cols:
        value_count1=df1[col].value_counts()
        value_count2=df2[col].value_counts()
        print(f'True:{value_count1} ')
    
        print(f'False:{value_count2} ')
    

In [42]:
#group by country name
c_grp=df.groupby(['Transported'])
True_grp=c_grp.get_group(True)
False_grp = c_grp.get_group(False)

In [43]:
value_counts(True_grp,False_grp,df_col_list)

True:HomePlanet
Earth     1951
Europa    1404
Mars       920
Name: count, dtype: int64 
False:HomePlanet
Earth     2651
Mars       839
Europa     727
Name: count, dtype: int64 
True:CryoSleep
True     2483
False    1789
Name: count, dtype: int64 
False:CryoSleep
False    3650
True      554
Name: count, dtype: int64 
True:Cabin
G/981/S    7
C/137/S    6
G/943/S    6
G/765/S    6
B/82/S     6
          ..
F/691/S    1
G/595/S    1
A/42/S     1
C/120/P    1
E/608/S    1
Name: count, Length: 3393, dtype: int64 
False:Cabin
G/974/P     6
E/13/S      6
F/1194/P    6
G/734/S     5
G/292/S     5
           ..
B/96/P      1
G/475/S     1
F/629/P     1
C/115/S     1
E/608/S     1
Name: count, Length: 3776, dtype: int64 
True:Destination
TRAPPIST-1e      2787
55 Cancri e      1098
PSO J318.5-22     401
Name: count, dtype: int64 
False:Destination
TRAPPIST-1e      3128
55 Cancri e       702
PSO J318.5-22     395
Name: count, dtype: int64 
True:Age
18.0    160
28.0    150
0.0     144
24.0    140
19

In [44]:
df.drop('PassengerId',axis='columns',inplace=True)



# just 2 feature

In [45]:
df_col_list=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported']


In [46]:
df_just2_f=df[['Cabin','Name','HomePlanet','CryoSleep','Destination','VIP']]
df_just2_f
y=df['Transported']

In [47]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='most_frequent')


In [48]:
df_just2_f=imputer.fit_transform(df_just2_f)

In [49]:
from sklearn.preprocessing import OrdinalEncoder
OrdinalEn=OrdinalEncoder()
df_just2_f_trans=OrdinalEn.fit_transform(df_just2_f)

In [50]:
df_just2_f_trans

array([[1.490e+02, 5.252e+03, 1.000e+00, 0.000e+00, 2.000e+00, 0.000e+00],
       [2.184e+03, 4.502e+03, 0.000e+00, 0.000e+00, 2.000e+00, 0.000e+00],
       [1.000e+00, 4.570e+02, 1.000e+00, 0.000e+00, 2.000e+00, 1.000e+00],
       ...,
       [5.285e+03, 3.002e+03, 0.000e+00, 0.000e+00, 2.000e+00, 0.000e+00],
       [2.131e+03, 1.596e+03, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
       [2.131e+03, 6.380e+03, 1.000e+00, 0.000e+00, 2.000e+00, 0.000e+00]])

In [51]:
y_prepared=list(map(int,y))
y_train=np.array(y_prepared)
y_train.reshape(-1)
y_train

array([0, 1, 0, ..., 1, 0, 1])

In [52]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [53]:
rnd_clf4=RandomForestClassifier()
cross_val_score(rnd_clf4,df_just2_f_trans,y_train,cv=3)

array([0.66908213, 0.61594203, 0.66724197])

# #finish

In [54]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes=='object':
            print(f'{column}:{df[column].unique()}')

In [55]:
print_unique_col_values(df)

HomePlanet:['Europa' 'Earth' 'Mars' nan]
CryoSleep:[False True nan]
Cabin:['B/0/P' 'F/0/S' 'A/0/S' ... 'G/1499/S' 'G/1500/S' 'E/608/S']
Destination:['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]
VIP:[False True nan]
Name:['Maham Ofracculy' 'Juanna Vines' 'Altark Susent' ... 'Fayey Connon'
 'Celeon Hontichre' 'Propsh Hontichre']


In [56]:
y=df['Transported']
y_prepared=list(map(int,y))
y_train=np.array(y_prepared)

In [57]:
y.value_counts()

Transported
True     4378
False    4315
Name: count, dtype: int64

In [58]:
X = df.drop(['Name','Transported'],axis='columns')

# Data Preprocessing

In [59]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer



In [60]:

num_pipeline=Pipeline(
[
    ('imputer',SimpleImputer(strategy='median')),
    ('std_scaler',StandardScaler())
])


In [61]:
cat_pipeline=Pipeline(
[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinal_encoder',OrdinalEncoder())
])




In [62]:
X = df.drop('Transported',axis='columns')

In [63]:
from sklearn.compose import ColumnTransformer

object_col=['HomePlanet','CryoSleep','Cabin','Destination','VIP','Name']
num_col=['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

full_pipeline= ColumnTransformer([
    ('num',num_pipeline,num_col),
    ('cat',cat_pipeline,object_col)
])

In [64]:
X_prepared=full_pipeline.fit_transform(X)

In [65]:
X_prepared

array([[ 7.11945365e-01, -3.33104659e-01, -2.81026731e-01, ...,
         2.00000000e+00,  0.00000000e+00,  5.25200000e+03],
       [-3.34037485e-01, -1.68073432e-01, -2.75386568e-01, ...,
         2.00000000e+00,  0.00000000e+00,  4.50200000e+03],
       [ 2.03685698e+00, -2.68000597e-01,  1.95999765e+00, ...,
         2.00000000e+00,  1.00000000e+00,  4.57000000e+02],
       ...,
       [-1.94573105e-01, -3.33104659e-01, -2.81026731e-01, ...,
         2.00000000e+00,  0.00000000e+00,  3.00200000e+03],
       [ 2.23820035e-01, -3.33104659e-01,  3.76365488e-01, ...,
         0.00000000e+00,  0.00000000e+00,  1.59600000e+03],
       [ 1.06060632e+00, -1.42334616e-01,  2.65687100e+00, ...,
         2.00000000e+00,  0.00000000e+00,  6.38000000e+03]])

# y prepared

In [66]:
y_prepared=list(map(int,y))
y_train=np.array(y_prepared)
y_train.reshape(-1)
y_train

array([0, 1, 0, ..., 1, 0, 1])

# Quick view of each  classifier performance

In [67]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression



In [68]:
svc_clf=SVC()
sgd_clf=SGDClassifier()
tree_clf=DecisionTreeClassifier()
rnd_clf=RandomForestClassifier()
log_clf=LogisticRegression()

In [69]:
from sklearn.model_selection import cross_val_score

models=[svc_clf,sgd_clf,tree_clf,rnd_clf,log_clf]
for model in models:
   acc=cross_val_score(model, X_prepared,y_train,cv=5,scoring="accuracy")
   print(f'model:{model}---->{acc}')

model:SVC()---->[0.58079356 0.56584244 0.54859114 0.56098964 0.58860759]
model:SGDClassifier()---->[0.49683726 0.50546291 0.50431282 0.49654776 0.50575374]
model:DecisionTreeClassifier()---->[0.72742956 0.72857964 0.71880391 0.71864212 0.75143843]
model:RandomForestClassifier()---->[0.7653824  0.76998275 0.77803335 0.80379747 0.79171461]
model:LogisticRegression()---->[0.77688327 0.77688327 0.76250719 0.77617952 0.79746835]


In [70]:
test_data=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test_data2= test_data.drop(['PassengerId'],axis='columns',)
test_data2

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...
4272,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [71]:
X_test_prepared=full_pipeline.fit_transform(test_data2)

# train model

In [72]:
rnd_clf.fit(X_prepared,y_train)

RandomForestClassifier()

In [73]:
y_pred=rnd_clf.predict(X_test_prepared)
y_pred

array([1, 0, 1, ..., 1, 1, 1])

In [74]:
y_pred=list(map(bool,y_pred))
y_pred=np.array(y_pred)
y_pred.reshape(-1)
y_pred


array([ True, False,  True, ...,  True,  True,  True])

In [75]:
df_sub=pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')


In [76]:
df_sub.columns



Index(['PassengerId', 'Transported'], dtype='object')

# sub

In [77]:
output = pd.DataFrame({'PassengerId': df_sub.PassengerId, 'Transported': y_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


# model 2

# Submission File

In [ ]:
output = pd.DataFrame({'PassengerId': df_sub.PassengerId, 'Transported': y_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

#  one subbmission done

# find important feature

In [83]:
feature=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']


In [ ]:
rnd_clf.fit(X_prepared,y_train)

In [ ]:
for name,score in zip(df3[feature],rnd_clf.feature_importances_):
    print(name,score)

In [ ]:
not_impf=['Cabin','VRDeck','RoomService','FoodCourt','Spa']
test_data3=test_data2.drop(not_impf,axis='columns')

In [ ]:
df3=df3.drop(not_impf,axis='columns')


In [ ]:
from sklearn.compose import ColumnTransformer

object_col=['HomePlanet','CryoSleep','Destination','VIP']
num_col=['Age','ShoppingMall']

full_pipeline= ColumnTransformer([
    ('num',num_pipeline,num_col),
    ('cat',cat_pipeline,object_col)
])




In [ ]:
X_test_prepared3=full_pipeline.fit_transform(test_data3)
X_train_prepared3=full_pipeline.fit_transform(df3)

In [ ]:
rnd_clf=RandomForestClassifier()
cross_val_score(rnd_clf,X_train_prepared3,y_train,cv=3,scoring="accuracy")

In [ ]:
rnd_clf2=RandomForestClassifier()
rnd_clf.fit(X_train_prepared3,y_train)


In [ ]:
y_pred= rnd_clf.predict(X_test_prepared3)

In [ ]:
def y_pred_fun(y_pred):
    y_pred=list(map(bool,y_pred))
    y_pred=np.array(y_pred)
    y_pred.reshape(-1)
    return y_pred

In [ ]:
output = pd.DataFrame({'PassengerId': df_sub.PassengerId, 'Transported': y_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

# log_clf  feeding importent feature

In [ ]:

log_clf2=LogisticRegression()
log_clf2.fit(X_train_prepared3,y_train)
y_pred=log_clf2.predict(X_test_prepared3)


In [ ]:
y_pred=y_pred_fun(y_pred)

In [ ]:
def submit_fun(df_sub,y_pred):
    output = pd.DataFrame({'PassengerId': df_sub.PassengerId, 'Transported': y_pred})
    output.to_csv('submission.csv', index=False)
    print("Your submission was successfully saved!")

In [ ]:
submit_fun(df_sub,y_pred)

# voting classifier

In [87]:
df.columns

Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name',
       'Transported'],
      dtype='object')

In [85]:
from sklearn.compose import ColumnTransformer

object_col=['HomePlanet','CryoSleep','Destination','VIP']
num_col=['Age','ShoppingMall']

full_pipeline= ColumnTransformer([
    ('num',num_pipeline,num_col),
    ('cat',cat_pipeline,object_col)
])

In [86]:
df_train=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
df_test=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
not_impf=['Cabin','VRDeck','RoomService','FoodCourt','Spa','']
df_train=df_train.drop(not_impf,axis='columns')
df_test=df_test.drop(not_impf,axis='columns')

#